In [ ]:
# default_exp signUp

# Sign up

> sign up

In [ ]:
#export
###################### Imports ###########################
import hashlib, uuid, os, logging, sys
import ujson as json
from awsSchema.apigateway import Event,Response
from src.passwordTable import PasswordTable
from beartype import beartype
from copy import deepcopy
from pynamodb.models import Model
from pynamodb.attributes import (
    UnicodeAttribute, NumberAttribute, UnicodeSetAttribute, UTCDateTimeAttribute
)

In [ ]:
#export
############### Logger for debugging code ##################
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

In [ ]:
#export
########## Error Definitions ##########
class HelperError(Exception): pass
class ParseInputError(HelperError): pass
class AddUserError(HelperError): pass
class CreateTableError(HelperError): pass
class UsernameAvailabilityError(HelperError): pass

In [ ]:
#hide
os.environ['USERPASSWORDTABLE'] = 'user-password-demo-sallee-master'

In [ ]:
#export
################ Setting Globals from Env Vars ################
USERPASSWORDTABLE = os.environ['USERPASSWORDTABLE']

In [ ]:
#export
########## Helper class for main function ##########
EventInput = dict
class H:
    @staticmethod
    @beartype
    def sha256(password: str) -> str:
        return hashlib.sha256(password.encode()).hexdigest()
    
    @staticmethod
    @beartype
    def salt() -> str:
        return uuid.uuid4().hex

    @classmethod
    @beartype
    def salted_sha256(cls, password: str, salt: str ='') -> tuple:
        if salt == '':
            salt = cls.salt()
        return f'{cls.sha256(salt + password)}', f'{salt}'
    
    @staticmethod
    @beartype
    def add_user_to_table(username: str, hash: str, salt: str, hashAndSalt: str):
        try:
            userTable = os.environ['USERPASSWORDTABLE']
            threadItem= PasswordTable(username=deepcopy(username), 
                                passwordHash=deepcopy(hash), 
                                salt=deepcopy(salt), 
                                hashAndSalt=deepcopy(hashAndSalt))
            threadItem.save()
        except Exception as e:
            logger.error(f'Unable to add user to the database:\n{e}')
            raise AddUserError(f'Unable to add user to the database:\n{e}')
        
    @staticmethod
    @beartype
    def parseInput(event: EventInput) -> tuple:
        '''
        returns username and password arguments from input
        '''
        
        body = Event.parseBody(deepcopy(event))
        logger.info(f'Event :: {body}')
        try:
            username = body['username']
        except KeyError:
            logger.error('username is not in body')
            raise ParseInputError('username is not in body')

        try:
            password = body['password']
        except KeyError:
            logger.error('password is not in body')
            raise ParseInputError('password is not in body')

        return username, password
    
    
    @staticmethod
    @beartype
    def usernameAvailable(username: str) -> bool:
        try:
            queryResult = PasswordTable.query(username)
            listResult = [row for row in queryResult]
            if len(listResult) > 0:
                return False
            return True
        except Exception as e:
            logger.error(f'Unable to check whether or not the username is available:\n{e}')
            raise UsernameAvailabilityError(f'Unable to check whether or not the username is available:\n{e}')
        

In [ ]:
#export
############################## Main Function ###############################
def signUp(event, *args):
    try:
        username, password = H.parseInput(event)
        hashedPw, salt = H.salted_sha256(password)
        hashAndSalt = hashedPw + ':' + salt
        if not H.usernameAvailable(username):
          raise UsernameAvailabilityError(f'User alreaady in database')
        H.add_user_to_table(username, hashedPw, salt, hashAndSalt)
        return Response.returnSuccess("Success")
    except Exception as e:
        return Response.returnError(f'Error :: {e}')